In [1]:
def MakeOne(aux,i,n): #i es Donde inicia, n es hasta donde lo hace
    for k in range(i,n):
        auxRow = aux[k][:]
        aux[k][:] = auxRow/aux[k][i]
    return aux

In [2]:
def Substract(aux,i,n): #i es Donde inicia, n es hasta donde lo hace
    for k in range(i,n):
        auxRow = aux[k][:] - aux[i-1][:]
        aux[k][:] = auxRow
    return aux

In [3]:
def Betas(mat,n):
    Betas = np.zeros([n+1,1])
    Betas[n] = 1
    for i in range(n-1,-1,-1):
        for j in range(n,i,-1):
            Betas[i] += mat[i][j] * Betas[j]
            #print(mat[i][j],B[j])
        Betas[i] *= (-1)
    #Betas[n] = (mat[n+1][n])
    return Betas


In [4]:
def Change_column_tensor(Mat, your_permutation): #Cambiar renglones para comparar si ya existen
    idx = np.empty_like(your_permutation)
    idx[:][:][your_permutation] = np.arange(len(your_permutation))
    return Mat[:,:, idx]

In [5]:
def Derivative(mat,bet): #mat es con binomios desarrollados  
    derBet = []
    for i in range(0,mat.shape[0]):
        for j in range(0,mat.shape[1]):
            if (mat[i][j][0] == 0 and mat[i][j][1] == 0 and mat[i][j][2] == 0):
                #print("Sin valores",mat[i][j])
                continue
                #break
            elif (mat[i][j][1] == float(bet) and float(bet)== mat[i][j][2]):
                #print("Variable al Cuadrado ",mat[i][j]," 2*mat[i][j][0],k,0")
                derBet.append(2*mat[i][j][0]) #[2*mat[i][j][0],mat[i][j][1],0]
                #break
            elif (mat[i][j][1] == float(bet) and mat[i][j][2] != -1.):
                #print("Una variable distinta ",mat[i][j]," mat[i][j][0],0,mat[i][j][2]")
                derBet.append(mat[i][j][0]) #[mat[i][j][0],0,mat[i][j][2]]
                #break
            elif (mat[i][j][2] == float(bet) and mat[i][j][1] != -1.):
                #print("Una variable distinta ",mat[i][j], " mat[i][j][0],mat[i][j][1],0")
                derBet.append(mat[i][j][0]) #[mat[i][j][0],mat[i][j][1],0]
                #break
            elif (mat[i][j][1] == float(bet) and mat[i][j][2] == -1.):
                #print("Solo la primer variable ",mat[i][j],"-- mat[i][j][0],0,0")
                derBet.append(-mat[i][j][0]) #[-mat[i][j][0],0,0]
                #break
            elif (mat[i][j][2] == float(bet) and mat[i][j][1] == -1.):
                #print("Solo la segunda variable ",mat[i][j],"-- mat[i][j][0],0,0")
                derBet.append(-mat[i][j][0]) #[-mat[i][j][0],0,0]
                #break
            elif (mat[i][j][2] == -1. and mat[i][j][1] == -1.):
                #print("Sin Variables ",mat[i][j], "0,0,0")
                continue
    return np.array(derBet)


In [6]:
def pre_Derivative(EquTotal, num):
    Mat = np.zeros((num,num+1))
    for i in range(0,num): #b numero columnas de matriz de ingreso que es iguala  al num de betas total
        Mat[i] = Derivative(EquTotal,i)
    return Mat

In [7]:
def Change_column_matrix(Mat, your_permutation): #Cambiar renglones para comparar si ya existen
    idx = np.empty_like(your_permutation)
    idx[:][:][your_permutation] = np.arange(len(your_permutation))
    return Mat[:, idx]

In [8]:
def left_shift_column(Mat): #Rotar las columnas a la izquierda
    left_shift = np.zeros(Mat.shape[1])
    for i in range(0,Mat.shape[1]):
        left_shift[i] = i-1
    left_shift = left_shift.astype(int)
    Mat_Total = Change_column_matrix(Mat,list(left_shift))
    #print(Mat_Total)
    return Mat_Total

In [9]:
def EquationSolver(mat,n): #n - numero de columnas, mat - Matriz que es el sistema de ecuaciones a resolver
    aux = mat
    for i in range(n-1):
        aux = MakeOne(aux,i,n)
        aux = Substract(aux,i+1,n)
    aux = MakeOne(aux,n-1,n)
    Beta = Betas(aux,n)
    return Beta


#El sistema ya resuelve 
#B[n] está de mas 

In [10]:
def Linear_Regression_Least_Square(mat):
    #print(mat.shape)
    mat_dev = Quadratic_Equation(mat)
    #print(mat)
    Mat = pre_Derivative(mat_dev,mat.shape[1])
    #print(Mat)
    Mat_Total = left_shift_column(Mat)
    #print(Mat_Total)
    Bet = EquationSolver(Mat_Total,Mat_Total.shape[0])
    return Bet


In [11]:
def Quadratic_Equation(Input_Matrix):
    Equ = np.zeros((Input_Matrix.shape[0],Input_Matrix.shape[1]+1)) #Crear matrix con dimensiones especificas
    for i in range(2,Input_Matrix.shape[1]+1):
        Equ.T[i] = Input_Matrix.T[i-2]
    Equ.T[0] = Input_Matrix.T[-1]
    Equ.T[1] = np.ones(Input_Matrix.shape[0])
    EquAux = np.zeros((Input_Matrix.shape[0] ,Input_Matrix.shape[1] + 1,3))
    for j in range(0,Input_Matrix.shape[1]+1):
        for i in range(0,Equ.shape[1]):  #Hacer Operaciones entre Betas
            if (i!=j):
                aux = sum(Equ.T[j]*Equ.T[i]*2)
                #aux = 1
                EquAux[j][i] = [aux,int(i-1),int(j-1)]
                #print(Equ.T[j],Equ.T[i])
            else:
                aux = sum(Equ.T[j]*Equ.T[i])
                #aux = 1
                #print(Equ.T[j],Equ.T[i])
                EquAux[j][i] = [aux,int(i-1),int(j-1)]
    EquAux2 = Change_column_tensor(EquAux,[0,2,1])
    for i in range(0,EquAux.shape[0]): #Funcion quitar Elemntos repetidos
        for j in range(0,EquAux.shape[1]):
            for k in range(0,EquAux.shape[0]):
                for h in range(0,EquAux.shape[1]):
                    comparison = EquAux[i][j] == EquAux2[k][h]
                    if (comparison.all()==True):
                        EquAux2[i][j] = [0,0,0]
    EquAux2 = Change_column_tensor(EquAux2,[0,2,1])
    EquTotal = EquAux - EquAux2
    return EquTotal

In [18]:
from time import time
import numpy as np
mat = np.array([[1,2,10],[5,4,5],[10,12,15],[15,17,7],[20,22,12]])
#mat = np.array([[1,10],[5,5],[10,15],[15,7],[20,12]])
#mat = np.array([[1,2,10,6,5],[5,4,5,6,5],[10,12,15,10,10],[15,17,7,9,11],[20,22,12,14,14],[22,24,15,12,13]])
#mat = np.random.randn(100,10)
tiempo_1 = time()
tiempo_inicial = time()
print(Linear_Regression_Least_Square(mat))

tiempo_final = time() 
tiempo_mat = tiempo_inicial - tiempo_1
tiempo_ejecucion = tiempo_final - tiempo_inicial
print(tiempo_mat)
print(tiempo_ejecucion)

[[ 8.1898575 ]
 [-2.8030176 ]
 [ 2.64920369]
 [ 1.        ]]
0.0
0.007995367050170898


In [20]:
git remote add origin https://github.com/Mausoleoo/Regression.git
git push -u origin master

SyntaxError: invalid syntax (<ipython-input-20-88b8f8c38481>, line 1)